In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import re
from tensorflow import keras
import numpy as np
from jamo import h2j, j2hcj
from tensorflow.keras.preprocessing.sequence import pad_sequences

c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\requests\__init__.py:78: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({0}) or chardet ({1}) doesn't match a supported "
c:\Users\choi seung un\AppData\Local\Programs\Python\Python3

In [85]:
path = "../../data/신한은행_거래내역조회_20230704112340.txt"

data = list()
with open(path, "rt", encoding="utf-8-sig") as f:
    while True:
        text = f.readline()[:-1]
        data.append(text)
        if not text:
            break

# 중복제거
data = list(set(data))

In [86]:
# 모델 불러오기
model_path = "../../model/model.h5"
model = keras.models.load_model(model_path)

In [76]:
# 특수문자 제거
def preprocessing(text):
    text_rmv = re.sub('[-=+,#/\?:^.@*\"※_~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', text)
    text_rmv = ' '.join(text_rmv.split())
    return text_rmv

In [92]:
def change_name(text):
    clean_text = preprocessing(text)
    tokens = clean_text.split()
    new_text = str()
    for token in tokens:
        # token 길이가 4인경우에 이름이 들어가는 경우는 거의 없는듯.
        if (len(token) == 3) or (len(token) >= 5):
            name_dic = dict()
            for i in range(len(token)-2):
                token_1 = token[i:i+3]
                embedding = np.array([[ord(i) for i in j2hcj(h2j(token_1))]])
                embedding = pad_sequences(embedding, maxlen=10, padding="post")
                result = model.predict(embedding, verbose=0)[0][1]
                # result = np.argmax(model.predict(embedding), axis=1)    
                name_dic[token_1] = result
            
            for i,n in name_dic.items():
                if n > 0.5:
                    token = token.replace(i, "[이름]")
        new_text += token + " "
    return new_text[:-1]



In [93]:
df = pd.DataFrame()
df["text"] = data
df["result"] = df["text"].progress_apply(lambda x: change_name(x))
df

100%|██████████| 248/248 [00:19<00:00, 12.78it/s]


,text,result
0,,
1,비플_미소돈까,비플 미소돈까
2,수도2208가정,수도2208가정
3,MICROSOFT*SUBS,MICROSOFT SUBS
4,(주)닉컴퍼니성과급,주 닉컴퍼니성과급
...,...,...
243,수도2304가정,수도2304가정
244,KT통신요금10,KT통신요금10
245,헤어살롱 H.L`d,헤어살롱 H L d
246,정소윤,[이름]


In [95]:
path = "../../data/result.csv"
df.to_csv(path, encoding="utf-8-sig", index=False)